# Assignment 2

### Leveraging NoSQL functionality in a RDBMS

In our last lecturer we have discovered that we can easily blend json and sql data using PostgreSQL's JSONB datatype and its operators.

Part 1.) Now it's your task to provide some answers using the migrated json data. 

Part 2.) Ingest data from an external service. Import data from the New York Times API

## Part 1.) Query jtracks data

Now that we have received information from our 3rd party seller. We would like to evaluate the data and generate some reports.

### Most popular artists
_Which artists have been listened to most often and how often?_ 

In [ ]:
select(body ->> 'name') as name,(body -> 'count')::INT as maxc
from jtracks
ORDER BY (body ->> 'count')::INT DESC
Limit 1;

### Generate an invoice report

The "counts" field represents the number a specific track has been listened to.
Based on the on the pricing information in the tracks table in, generate a report that calculates the total value per track.

One count equals one single order. 


Further Explanation: If the price of a single track is 0.99 and its count equals 10, the total equals 9.90.

In [ ]:
SELECT DISTINCT subq.name, subq.unit_price * (jtracks.body->'count')::INT as total
FROM(SELECT *
     FROM tracks t join albums a on t.album_id = a.id ) as subq
join jtracks on (subq.name = jtracks.body->>'name');
--WHERE jtracks.body->>'album' = subq.title;

### Most valued artists

Similar to the previous question but grouped by the corresponding artist: Using the some information in jtracks, group the total amount by the artist name.

Your result should look like:

| artist_name | total | 
|-------------|-------|
| AC/DC       | 312.23|
| Rammstein   | 200.21|
| Queens      |  33.12| 

In [ ]:
SELECT  artists.name,sum(query.total)
FROM (  SELECT DISTINCT subq.artist_id,subq.name, subq.title, subq.unit_price * (jtracks.body->'count')::INT as total
FROM( SELECT *
      FROM tracks t join albums a on t.album_id = a.id ) as subq
join jtracks on (subq.name = jtracks.body->>'name' )
) as query
--WHERE jtracks.body->>'album' LIKE subq.title) as query
join artists on query.artist_id = artists.id
group by artists.name;

### Missing talent

There are some artists that are not in our music-store. Of those missing in our music-store, figure out the ones that should be signed us.

Query the top 5 artists that have the most total count and are not in our music store. Order your result descending.

In [ ]:
SELECT body->>'name' as name, body->>'album' as album, body->>'artist'as artist, (body->'count')::INT as count
FROM jtracks
where body->>'name' NOT IN (SELECT name
FROM tracks)
ORDER BY count desc
LIMIT 5;

## Part 2.) Data Ingestion from NYT API

We ask you to write an application that fetches data from the NYT API (https://developer.nytimes.com/apis) and subsequently run some queries against your dataset.
In order to be able to access the NYT API, you'll have to set up an account. After setting up your account, you should be able to fetch data from the REST API using the provided api key. 
In order to figure out how to provide answers to the following tasks, you'll have to work through to API documentation on your own.
You are allowed to use any python 3rd party libraries you want. Be sure to upload this notebook and your python source code + requirements.txt to moodle. 
Carefully think whether you want to store the whole data returned from an api request or only a fraction of it. 
You will either have to store the data in your docker database or in a custom database. (This will be decided by the end of the last lecture).

You shall provide the sql query that answers the following questions

### Number of most viewed
How many most viewed articles have been there within the last 30 days?

In [ ]:
SELECT count(*)
FROM ds22m017_viewed;

### Section and subsection of most viewed articles

Of all most viewed articles within the last 30 days, what where the numbers of articles per section and subsection?


Your result should look like the following:


| Section | subsection | number | 
|------------|---------|--------|
| U.S.       |Politics |5       |
| U.S.       |         |2       |
| Business   |         |3       |
| Opinion    |         |2       | 

In [ ]:
SELECT ds22m017_viewed.body ->> 'section' as section, ds22m017_viewed.body ->> 'subsection' as subsection,count(*) as number
FROM ds22m017_viewed join ds22m017_emailed d22m017e on ds22m017_viewed.body ->> 'section' = d22m017e.body ->> 'section'
GROUP BY section, subsection;

## Most viewed vs most shared 

Of all most viewed and most shared sections, compare the number of shared and viewed articles per section within the last 30 days.


Your result shall look like the following: (notice '0' if a section does not occur within a type)


| Section    | type    | number | 
|------------|---------|--------|
| U.S.       |shared   |5       |
| U.S.       |viewed   |3       |
| Opinion    |shared   |2       | 
| Opinion    |viewed   |0       | 

In [ ]:

select sections.section, 'shared' as type, count(body->>'id')
from
ds22m017_emailed
    right join
(select body->>'section' as section
from ds22m017_viewed
union
select body->>'section' as section
from ds22m017_emailed) as sections
on ds22m017_emailed.body->>'section' = sections.section
group by section
union
select sections.section, 'viewed' as type, count(body->>'id')
from
ds22m017_viewed
    right join
(select body->>'section' as section
from ds22m017_viewed
union
select body->>'section' as section
from ds22m017_emailed) as sections
on ds22m017_viewed.body->>'section' = sections.section
group by section
order by section, type

### Be creative

Last but not least, we want you to think of any interesting information you'd like to evaluate using the NYT API.
What kind of information might be interesting to know? - You don't need to put too much thought into the question you'd like to answer, but be creative!

There's only one catch! - You must use a different category, e.g. top stories, books, movies,... 

The question you would like to answer = How often do the different ratings appear in the movies?

In [ ]:
SELECT d.body->>'mpaa_rating' as rating, COUNT(*)
FROM ds22m017_movies d
group by rating;